In [ ]:
#coding=utf-8

from flask import Flask,jsonify,render_template,request,g,current_app
from py2neo import *
from functools import reduce
import query
import image_data
import tupu

app = Flask(__name__)
graph = Graph("http://localhost:7474",auth=("neo4j","123456"))



@app.route('/',methods = ['GET','POST'])
def index():
    if request.method == 'GET':
        image_list = []      #POST前输入index.html的图片参数先设空值
        image_record = []
        data = {}
        #elements = tupu.graph_func()
        return render_template('index.html',image_list = image_list,image_record = image_record,data = data)
        
    if request.method == 'POST':
        chaodai = request.form.get("chaodai")     #获取朝代表单值
        dongku = request.form.get("dongku")
        xingzhi = request.form.get("xingzhi")
        zhuti = request.form.get("zhuti")
        buju = request.form.get("buju")
        leixing = request.form.get("leixing")
        wuding = request.form.get("wuding")
        wudingjiegoufujian = request.form.get("wudingjiegoufujian")
        washi = request.form.get("washi")
        wushen = request.form.get("wushen")
        wushenjiegoufujian = request.form.get("wushenjiegoufujian")
        qiangshi = request.form.get("qiangshi")
        jizuo = request.form.get("jizuo")
        jizuofujian = request.form.get("jizuofujian")
        
        #所有表单提交查询列表
        click = [chaodai,dongku,xingzhi,zhuti,buju,leixing,wuding,wudingjiegoufujian,washi,wushen,
              wushenjiegoufujian,qiangshi,jizuo,jizuofujian]
        
        g.click = click       #click化为G对象        
        cypher = query.query_func()  #G对象作为容器,将参数click传递到query模块query_func()函数，调用后得到Cypher问句
        #print(cypher)
        
        result = graph.run(cypher).data()         #运行Cypher问句
        print(result)
        g.result = result       #运行结果化为G对象
        #get_graph()     #get_graph()中才能调用g.result，不会AttributeError: '_AppCtxGlobals' object has no attribute 'result'
        
        out = image_data.image_func()          #调用image_data模块image_func()函数中，得到图像路径与名称
        #print(out)
        image_list = out[0]
        image_record = out[1]
        data = out[2]
        
        #elements = tupu.graph_func()    #graph为GET方法
                
        return  render_template('index.html',image_list = image_list,image_record = image_record,data = data)

'''
#由code.js中的$.get('/graph', function(result_nr) {}, 'json')调用
#把neo4j的数据转成符合cytoscape要求的JSON数据存在code.js的result_nr中
@app.route('/graph')
def get_graph():
    g.input = g.result    #AttributeError: '_AppCtxGlobals' object has no attribute 'result'
    elements = tupu.graph_func()         #调用tupu模块的graph_func函数，将json格式的节点、关系返回前端知识图谱cy中
    #print(nodes)
    #print(edges)
    
    #print(g.result)

    #nodes = [{'data': {'id': '172', 'name': 'Tom Cruise', 'label': 'Person'}},
      #          {'data': {'id': '183', 'title': 'Top Gun', 'label': 'Movie'}},
      #          {'data': {'id': '200', 'name': 'AAA', 'label': 'Movie'}}]
    #edges = [{'data': {'source': '172', 'target': '183', 'relationship': 'Acted_In'}}]
    
    
    #return jsonify(elements = {"nodes": nodes, "edges": edges})    #用jsonify函数把elements转成JSON格式返回给前端
    return elements
'''

if __name__ == '__main__':
    app.run()
    


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Feb/2022 09:01:12] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Feb/2022 09:01:12] "GET /graph HTTP/1.1" 404 -


In [ ]:
from flask import Flask,g

app = Flask(__name__)

@app.before_request
def bfr():
    g.name = 'jack'

@app.route('/')
def index1():
    print(g.name)
    return('index1页面')


@app.route('/index2')
def index2():
    print(g.name)
    return('index2页面')

if __name__ == '__main__':
    app.run()


In [52]:
'''

app = Flask(__name__)
graph = Graph()

def buildNodes(nodeRecord):
    data = {"id": str(nodeRecord['id']), "label": next(iter(nodeRecord['label']))}
    data.update(nodeRecord['n'])
    return {"data": data}

def buildEdges(relationRecord):
    data = {"source": str(relationRecord['start_id']), "target": str(relationRecord['end_id']), "relationship": relationRecord['type']}
    return {"data": data}

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/graph')
def get_graph():
    nodes = list(map(buildNodes, py2neo.client.json.dehydrate(graph.run('MATCH (n) RETURN id(n) as id, labels(n) as label, n').data())))
    edges = list(map(buildEdges, py2neo.client.json.dehydrate(graph.run('MATCH (n1)-[r]->(n2) RETURN id(r) as id, id(n1) as start_id, id(n2) as end_id, type(r) as type, r').data())))
    return jsonify(elements = {"nodes": nodes, "edges": edges})

if __name__ == '__main__':
    app.jinja_env.auto_reload = True
    app.config['TEMPLATES_AUTO_RELOAD'] = True
    app.run(debug = True, host='10.110.41.71')
    
'''

'\n\napp = Flask(__name__)\ngraph = Graph()\n\ndef buildNodes(nodeRecord):\n    data = {"id": str(nodeRecord[\'id\']), "label": next(iter(nodeRecord[\'label\']))}\n    data.update(nodeRecord[\'n\'])\n    return {"data": data}\n\ndef buildEdges(relationRecord):\n    data = {"source": str(relationRecord[\'start_id\']), "target": str(relationRecord[\'end_id\']), "relationship": relationRecord[\'type\']}\n    return {"data": data}\n\n@app.route(\'/\')\ndef index():\n    return render_template(\'index.html\')\n\n@app.route(\'/graph\')\ndef get_graph():\n    nodes = list(map(buildNodes, py2neo.client.json.dehydrate(graph.run(\'MATCH (n) RETURN id(n) as id, labels(n) as label, n\').data())))\n    edges = list(map(buildEdges, py2neo.client.json.dehydrate(graph.run(\'MATCH (n1)-[r]->(n2) RETURN id(r) as id, id(n1) as start_id, id(n2) as end_id, type(r) as type, r\').data())))\n    return jsonify(elements = {"nodes": nodes, "edges": edges})\n\nif __name__ == \'__main__\':\n    app.jinja_env.auto